In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
import numpy as np
import random 
import string
import speech_recognition as sr

In [2]:
r = sr.Recognizer()
mic=sr.Microphone()
f=open(r'Corpus_crucero.txt','r',errors='ignore')
texto=f.read()
texto

'Para saber cómo vestir, la estación del año y la temperatura media de la zona marcan siempre la vestimenta más adecuada para los cruceros, aunque los cruceros tienden a ser cada vez más informales sobre todo durante el día.\n\nEl programa del día le indicará siempre la ropa más adecuada.\n\nRespecto al calzado, en tierra necesitarás zapatos cómodos para caminar, así como sombrero y protección solar (en verano).\n\nLas propinas son una norma generalizada, obligatorias, en todos los barcos de crucero, de hecho, forman parte del sueldo del personal y forman parte del llamado servicio.\n\nDependiendo de la naviera, para embarcar es conveniente llegar al embarque 3 o 4 horas antes de la hora de zarpar, normalmente los embarques suelen ser al mediodía y casi todas las compañías de cruceros ofrecen un buffet para comer.\n\nLa comida es un punto fuerte, los cruceros son una oportunidad única para poder degustar una amplia gama de alta calidad, con una diversidad de platos típicos de todas las

# Preprocesamiento del corpus

In [3]:
texto=texto.lower() #convertir en minuscula
#instalar punkt y wordnet
sent_tokens= nltk.sent_tokenize(texto) #tokenizo por sentencias
word_tokens=nltk.word_tokenize(texto) #tokenizo por palabras
lematizador= WordNetLemmatizer()

def LemTokens(tokens):
    return [lematizador.lemmatize(token) for token in tokens] #lematiza las palabras sin simbolos (acentos..)

removerSimb= dict((ord(simbolo),None) for simbolo in string.punctuation)
def LemNormalizar(texto):
    return LemTokens(nltk.word_tokenize(texto.lower().translate(removerSimb)))

# Preprocesamiento del mensaje y evaluar similitud

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords

In [5]:
def respuesta(entrada_user):
    chat_respuesta=''
    lenguaje=stopwords.words('spanish')
    sent_tokens.append(entrada_user) #añade al final del corpus la pregunta del usuario.
    finversa= TfidfVectorizer(tokenizer=LemNormalizar,stop_words=lenguaje)
    finversaTrain=finversa.fit_transform(sent_tokens)
    #evalua la similitud entre el corpus y el mensaje (finversaTrain)
    valorSimilitud = cosine_similarity(finversaTrain[-1],finversaTrain)
    indice = valorSimilitud.argsort()[0][-2] #devuelve el indice del anteultimo elemento ordenado (respuesta)
    flat = valorSimilitud.flatten() #convierte en array
    flat.sort() #lo ordeno
    respuesta = flat[-2]
    if respuesta ==0:
        chat_respuesta=chat_respuesta + ' no se que responderte, comunicate con martin@crucero.com '
        return chat_respuesta
    else:
        chat_respuesta=chat_respuesta + sent_tokens[indice]
        return chat_respuesta
    

In [ ]:
#respuesta('el desayuno como es')

# definicion de coincidencias

In [6]:
saludo_entrada_usuario=["hola","buenas","saludos","hey","como va"]
saludo_salida_chatbot=["hola","hola que tal?","hola ¿Como te puedo ayudar?","hola encantado en conocerte"]

def saludos(sentencia):
    for word in sentencia.split():
        if word.lower() in saludo_entrada_usuario:
            return random.choice(saludo_salida_chatbot)

# reconocimiento de voz

In [7]:
def reconocerVoz():
    with mic as source:
    #r.adjust_for_ambient_noise(source)
        audioGrabado=r.listen(source)
    try:
        texto=r.recognize_google(audioGrabado,language='es-ES') 
    except sr.UnknownValueError:
        texto="Consulta: "
    return texto

# generación de respuesta

In [8]:
flag=True
print('MartinBot: mi nombre es MartinBot. Contestaré tus dudas sobre el crucero. Si queres salir escribi salir ')
while(flag==True):
    #pregunta=input()
    pregunta=reconocerVoz()
    if pregunta!="Consulta: ":
        pregunta=pregunta.lower() #convertir a minuscula
        if(pregunta!='salir'):
            respuestaChatBot=saludos(pregunta)
            if respuestaChatBot!=None:
                print("MartinBot: " +respuestaChatBot)
            else:
                respuestaChatBot=respuesta(pregunta)
                print("MartinBot: " +respuestaChatBot)
                sent_tokens.remove(pregunta)
        else:
            flag=False
            print("MartinBot: Muchas gracias por usar el servicio, saludos" )
    

MartinBot: mi nombre es MartinBot. Contestaré tus dudas sobre el crucero. Si queres salir escribi salir 
MartinBot: está prohibido fumar en los teatros, en los restaurantes y buffet de autoservicio (locales cerrados), en los corredores, pasillos, vestíbulos, antesalas e interiores de los ascensores y escaleras.
MartinBot: los barcos de cruceros disponen de un servicio médico en caso de que se ponga enfermo, siendo los servicios médicos prestados son de pago, por lo que recomendamos utilizar un seguro médico.
MartinBot: la mayoría de buques ya disponen de internet como servicio.


KeyboardInterrupt: 